### Minimal LineageOT demo

This notebook shows a minimal working example of the LineageOT pipeline.

In [1]:
import anndata
import lineageot
import numpy as np

In [2]:
rng = np.random.default_rng()

The anndata object requires three kinds of information:

- Cell state measurements (in adata.X or adata.obsm)
- Cell sampling times, relative to the root of lineage tree, i.e. fertilization (in adata.obs)
- Cell lineage barcodes (in adata.obsm)

The barcodes should be encoded as row vectors where each entry corresponds to a possibly-mutated site. A positive number indicates an observed mutation, zero indicates no mutation, and -1 indicates the site was not observed.

For example, if row `i` of `adata.obsm['barcodes']` is
```
[0, 0, 13, -1]
```
that means that, out of four possible sites for mutations, cell `i` was observed to have no mutations in the first two sites and mutation 13 in the third site. The last site was not observed for cell `i`.

In [3]:
#Creating a minimal fake AnnData object to run LineageOT on

t1 = 5;
t2 = 10;

n_cells_1 = 5;
n_cells_2 = 10;
n_cells = n_cells_1 + n_cells_2;

n_genes = 5;

barcode_length = 10;

adata = anndata.AnnData(X = np.random.rand(n_cells, n_genes),
                        obs = {"time" : np.concatenate([t1*np.ones(n_cells_1), t2*np.ones(n_cells_2)])},
                        obsm = {"barcodes" : rng.integers(low = -1, high = 10, size = (n_cells, barcode_length))}
                       )

In [4]:
# Running LineageOT

lineage_tree_t2 = lineageot.fit_tree(adata[adata.obs['time'] == t2], t2)
coupling = lineageot.fit_lineage_coupling(adata, t1, t2, lineage_tree_t2)

     pcost       dcost       gap    pres   dres
 0: -1.6721e+02 -2.7373e+02  1e+02  0e+00  6e-01
 1: -1.8256e+02 -1.9172e+02  9e+00  8e-17  8e-02
 2: -1.8461e+02 -1.8518e+02  6e-01  8e-17  3e-03
 3: -1.8462e+02 -1.8463e+02  1e-02  8e-17  4e-05
 4: -1.8462e+02 -1.8462e+02  1e-04  8e-17  4e-07
 5: -1.8462e+02 -1.8462e+02  1e-06  5e-17  4e-09
Optimal solution found.


In [5]:
# Saving the fitted coupling in the format Waddington-OT expects
lineageot.save_coupling_as_tmap(coupling, t1, t2, './tmaps/example')